In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
from natsort import natsorted, ns
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
import keras
from PIL import Image
import cv2
from keras import models
from keras.layers import Conv2D, MaxPooling2D, Dense, Activation, BatchNormalization, Input, Dropout, Flatten, Concatenate
from keras.models import Model, Sequential
from keras import backend as K
from keras.layers import Lambda
from keras import optimizers
from keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve
from tensorflow.keras import initializers

In [50]:
PATH_ORG = "/content/drive/MyDrive/Dataset/Signature_Set1/genuine"
PATH_FORG = "/content/drive/MyDrive/Dataset/Signature_Set1/forged"

In [46]:
PATH_ORG = "/content/drive/MyDrive/Dataset/Signature_Set2/full_org"
PATH_FORG = "/content/drive/MyDrive/Dataset/Signature_Set2/full_forg"

In [51]:
def get_list(path_o, path_fo):
  org_path = os.listdir(path_o)
  forg_path = os.listdir(path_fo)
  org_imgs = [path for path in org_path if path.endswith(".png")]
  forg_imgs = [path for path in forg_path if path.endswith(".png")]
  org_imgs = natsorted(org_imgs, alg=ns.IGNORECASE)
  forg_imgs = natsorted(forg_imgs, alg=ns.IGNORECASE)
  return org_imgs, forg_imgs
 

In [52]:
org_imgs, forg_imgs = get_list(PATH_ORG, PATH_FORG)

In [ ]:
org_imgs

In [54]:
print(len(org_imgs))
print(len(forg_imgs))

120
120


In [56]:
def create_df(originals,forgeries, path_o, path_fo):
  image_1_o, image_2_o, label_o = [], [], []
  image_1_f, image_2_f, label_f = [], [], []

  # for writer_nr in range(55): # for the first dataset
  #   for sig_A_nr in range(24):
  #     for sig_B_nr in range(24):
  #       image_1_o.append(os.path.join(path_o, originals[writer_nr*24 + sig_A_nr]))
  #       image_2_o.append(os.path.join(path_o, originals[writer_nr*24 + sig_B_nr]))
  #       label_o.append(0)

  for writer_nr in range(24): # for the second dataset
    for sig_A_nr in range(5):
      for sig_B_nr in range(5):
        image_1_o.append(os.path.join(path_o, originals[writer_nr*5 + sig_A_nr]))
        image_2_o.append(os.path.join(path_o, originals[writer_nr*5 + sig_B_nr]))
        label_o.append(0)

  # for writer_nr in range(55): # for the first dataset
  #   for sig_A_nr in range(24):
  #     for sig_B_nr in range(24):
  #       image_1_f.append(os.path.join(path_o, originals[writer_nr*24 + sig_A_nr]))
  #       image_2_f.append(os.path.join(path_fo, forgeries[writer_nr*24 + sig_B_nr]))
  #       label_f.append(1)

  for writer_nr in range(24): 
    for sig_A_nr in range(5):
      for sig_B_nr in range(5):
        image_1_f.append(os.path.join(path_o, originals[writer_nr*5 + sig_A_nr]))
        image_2_f.append(os.path.join(path_fo, forgeries[writer_nr*5 + sig_B_nr]))
        label_f.append(1)
  
  image_1 = image_1_o + image_1_f
  image_2 = image_2_o + image_2_f
  label = label_o + label_f

  data = {"image_1": image_1, "image_2": image_2, "label": label}
  df = pd.DataFrame(data, columns = ["image_1","image_2","label"])
  #df=df.reindex(np.random.permutation(df.index))
  return df

In [57]:
df = create_df(org_imgs, forg_imgs, PATH_ORG, PATH_FORG)
print(df.shape)
print(df.iloc[73,0])

(1200, 3)
/content/drive/MyDrive/Dataset/Signature_Set1/genuine/NFI-00305003.png


In [58]:
# df_test = df[0:3456] # 6912/63360=10,9%
# df_test2 = df[31680:35136]
# df_test=df_test.append(df_test2, ignore_index=True)
# df_test.shape

df_test = df[0:75] # 150/1200 = 12,5%
df_test2 = df[600:675]
df_test=df_test.append(df_test2, ignore_index=True)
df_test.shape

(150, 3)

In [59]:
# df_val = df[3456:8640] # 10368/63360=16,36%
# df_val2 = df[35136:40320]
# df_val=df_val.append(df_val2, ignore_index=True)
# df_val.shape

df_val = df[75:175] # 200/1200 = 16,7%
df_val2 = df[675:775]
df_val=df_val.append(df_val2, ignore_index=True)
df_val.shape

(200, 3)

In [60]:
# df1 = df[8640:31680] # 46080/63360=72,72%
# df2 = df[40320:]
# df_train=df1.append(df2, ignore_index=True)
# df_train.shape

df1 = df[175:600] # 850/1200=70,8%
df2 = df[775:]
df_train=df1.append(df2, ignore_index=True)
df_train.shape

(850, 3)

In [61]:
df_train = df_train.sample(frac = 1) 
df_test = df_test.sample(frac = 1) 
df_val = df_val.sample(frac = 1) 
print(df_test.head)

<bound method NDFrame.head of                                                image_1  ... label
81   /content/drive/MyDrive/Dataset/Signature_Set1/...  ...     1
82   /content/drive/MyDrive/Dataset/Signature_Set1/...  ...     1
32   /content/drive/MyDrive/Dataset/Signature_Set1/...  ...     0
102  /content/drive/MyDrive/Dataset/Signature_Set1/...  ...     1
23   /content/drive/MyDrive/Dataset/Signature_Set1/...  ...     0
..                                                 ...  ...   ...
127  /content/drive/MyDrive/Dataset/Signature_Set1/...  ...     1
99   /content/drive/MyDrive/Dataset/Signature_Set1/...  ...     1
56   /content/drive/MyDrive/Dataset/Signature_Set1/...  ...     0
103  /content/drive/MyDrive/Dataset/Signature_Set1/...  ...     1
19   /content/drive/MyDrive/Dataset/Signature_Set1/...  ...     0

[150 rows x 3 columns]>


In [ ]:
nr= 603
print(df.iloc[nr,0])
print(df.iloc[nr,1])
print(df.iloc[nr,2])

/content/drive/MyDrive/Dataset/Signature_Set1/genuine/NFI-00101001.png
/content/drive/MyDrive/Dataset/Signature_Set1/forged/NFI-00102019.png
1


In [13]:
class DataGenerator(keras.utils.Sequence):
    
    def __init__(self, df, batch_size=32, dim=(71,110), n_channels=3, shuffle=False):
        self.dim = dim
        self.batch_size = batch_size
        self.df = df
        self.labels = df["label"]
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()
 
    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))
 
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        rows = [self.df.iloc[k] for k in indexes]
        X, y = self.__data_generation(rows)
        return X, y
 
    def on_epoch_end(self):
        self.indexes = np.arange(self.df.shape[0])
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
 
    def __data_generation(self, rows):
        x_1 = np.empty((self.batch_size, *self.dim, self.n_channels))
        x_2 = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=float)
        
        for i in range(len(rows)):
            image_1 = cv2.imread(rows[i]["image_1"])
            image_1 = cv2.resize(image_1,(110,72),cv2.INTER_AREA)
            image_1 = cv2.bitwise_not(image_1)
            image_1=np.array(image_1)
            image_2 = cv2.imread(rows[i]["image_2"])
            image_2 = cv2.resize(image_2,(110,72), cv2.INTER_AREA)
            image_2 = cv2.bitwise_not(image_2)
            image_2=np.array(image_2)
          
            x_1[i,] = image_1/255
            x_2[i,] = image_2/255
            y[i] = rows[i]["label"]
 
        return [x_1, x_2], y

In [ ]:
df_test.to_csv('out.csv', index=False)

In [ ]:
#https://www.kaggle.com/arpandhatt/siamese-neural-networks
# We have 2 inputs, 1 for each picture
left_input = Input((110,72,3))
right_input = Input((110,72,3))

# We will use 2 instances of 1 network for this task
convnet = Sequential([
    Conv2D(5,3, input_shape=(110,72,3)),
    Activation('relu'),
    MaxPooling2D(),
    Conv2D(5,3),
    Dropout(0.5),
    Activation('relu'),
    MaxPooling2D(),
    Conv2D(7,2),
    Dropout(0.5),
    Activation('relu'),
    MaxPooling2D(),
    Conv2D(7,2),
    Dropout(0.5),
    Activation('relu'),
    Flatten(),
    Dense(18),
    Activation('sigmoid')
])
# Connect each 'leg' of the network to each input
# Remember, they have the same weights
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)

# Getting the L1 Distance between the 2 encodings
L1_layer = Lambda(lambda tensor:K.abs(tensor[0] - tensor[1]))

# Add the distance function to the network
L1_distance = L1_layer([encoded_l, encoded_r])

prediction = Dense(1,activation='sigmoid')(L1_distance)
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)

optimizer = Adam(0.001, decay=2.5e-4)
#//TODO: get layerwise learning rates and momentum annealing scheme described in paperworking
siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=['accuracy'])

In [ ]:
model.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 110, 72, 3)] 0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 110, 72, 3)] 0                                            
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 18)           9054        input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 18)           0           sequential_3[0][0]    

In [17]:
 params={
    'dim': (72,110),
    'batch_size': 32,
    'n_channels': 3,
    'shuffle': False
}

In [20]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='model.{epoch:02d}.h5', verbose=1, save_best_only=False, save_weights_only=False, save_freq='epoch')

In [16]:
steps=len(df_train)/32
val_steps = len(df_val)/32

train_datagen = DataGenerator(df_train,**params)
validation_datagen = DataGenerator(df_val,**params)
siamese_net.fit(train_datagen,validation_data=validation_datagen, epochs=10, steps_per_epoch=steps, validation_steps=val_steps, use_multiprocessing=True, workers=6, callbacks=[checkpointer])

Epoch 1/10
1440/1440 [==============================] - ETA: 0s - loss: 0.1886 - accuracy: 0.9152WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00001: saving model to model.01.h5
1440/1440 [==============================] - 522s 362ms/step - loss: 0.1886 - accuracy: 0.9152 - val_loss: 0.1458 - val_accuracy: 0.9770
Epoch 2/10
1440/1440 [==============================] - ETA: 0s - loss: 0.0121 - accuracy: 1.0000WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00002: saving model to model.02.h5
1440/1440 [==============================] - 451s 313ms/step - loss: 0.0121 - accuracy: 1.0000 - val_loss: 0.1420 - val_accuracy: 0.9633
Epoch 3/10
1439/1440 [============================>.] - ETA: 0s - loss: 0.0064 - accuracy: 1.0000WARNING:tensorflo

In [23]:
from keras.models import load_model

In [22]:
model = load_model('/content/model.01.h5')
params={
    'dim': (72,110),
    'batch_size': 32,
    'n_channels': 3,
    'shuffle': False
}
checkpointer = ModelCheckpoint(filepath='model2.{epoch:02d}.h5', verbose=1, save_best_only=False, save_weights_only=False, save_freq='epoch')
steps=len(df_train)/32
val_steps = len(df_val)/32

train_datagen = DataGenerator(df_train,**params)
validation_datagen = DataGenerator(df_val,**params)
model.fit(train_datagen,validation_data=validation_datagen, epochs=10, steps_per_epoch=steps, validation_steps=val_steps, use_multiprocessing=True, workers=6, callbacks=[checkpointer])

Epoch 1/10
27/26 [==============================] - ETA: 0s - loss: 0.7392 - accuracy: 0.6100WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00001: saving model to model2.01.h5
27/26 [==============================] - 30s 1s/step - loss: 0.7392 - accuracy: 0.6100 - val_loss: 0.5359 - val_accuracy: 0.7277
Epoch 2/10
27/26 [==============================] - ETA: 0s - loss: 0.5176 - accuracy: 0.7303WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00002: saving model to model2.02.h5
27/26 [==============================] - 27s 998ms/step - loss: 0.5176 - accuracy: 0.7303 - val_loss: 0.5814 - val_accuracy: 0.7054
Epoch 3/10
27/26 [==============================] - ETA: 0s - loss: 0.3919 - accuracy: 0.8391WARNING:tensorflow:multiprocessing can i

In [ ]:
model = load_model('/content/model2.10.h5')

In [36]:
test_datagen = DataGenerator(df_test,**params) # first dataset
loss, acc = model.evaluate(test_datagen,steps=3, verbose=1)

3/3 [==============================] - 8s 3s/step - loss: 0.1781 - accuracy: 1.0000


In [62]:
test_datagen = DataGenerator(df_test,**params) # second dataset
loss, acc = model.evaluate(test_datagen,steps=3, verbose=1)

3/3 [==============================] - 1s 282ms/step - loss: 0.3435 - accuracy: 0.8229


In [66]:
im_1 = cv2.imread("/content/drive/MyDrive/Dataset/Signature_Set1/forged/NFI-00101014.png")
im_2 = cv2.imread("/content/drive/MyDrive/Dataset/Signature_Set1/genuine/NFI-00103001.png")  
im_1 = cv2.resize(im_1,(110,71))
im_2 = cv2.resize(im_2,(110,71))
im_1 = cv2.bitwise_not(im_1)
im_2 = cv2.bitwise_not(im_2)
im_1 = im_1/255
im_2 = im_2/255
im_1 = np.expand_dims(im_1,axis=0)
im_2 = np.expand_dims(im_2,axis=0)
 
y_pred = model.predict([im_1,im_2])
if y_pred < 0.5:
  wynik = "oryginał"
else:
  wynik = "podrobiony"
print(y_pred, wynik)

[[0.9945199]] podrobiony
